# 🔐 NeMo Safe Synthesizer: Advanced Privacy (Differential Privacy)

> ⚠️ **Warning**: NeMo Safe Synthesizer is in Early Access and not recommended for production use.

<br>

In this notebook, we create synthetic tabular data using the NeMo Microservices Python SDK with differential privacy enabled. The notebook should take about 1.5 hours to run.

After completing this notebook, you'll be able to:
- **Use the NeMo Microservices SDK** to interact with Safe Synthesizer
- **Enable differential privacy** to provide additional privacy protection
- **Access an evaluation report** on the quality and privacy of the synthetic data

#### 💾 Install dependencies

Ensure you have a NeMo Microservices Platform deployment available. If you're using a managed or remote deployment, have the correct base URLs and tokens ready.

In [ ]:
import pandas as pd
from nemo_microservices import NeMoMicroservices
from nemo_microservices.beta.safe_synthesizer.builder import SafeSynthesizerBuilder

import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

### ⚙️ Initialize the NeMo Safe Synthesizer Client

- The Python SDK provides a wrapper around the NeMo Microservices Platform APIs.
- `http://localhost:8080` is the default URL for `base_url` in quickstart.
- If using a managed or remote deployment, ensure you use the correct base URLs and tokens.

In [ ]:
client = NeMoMicroservices(
    base_url="http://localhost:8080",
)

NeMo DataStore is launched as one of the services. We'll use it to manage storage, so set the following:

In [ ]:
datastore_config = {
    "endpoint": "http://localhost:3000/v1/hf",
    "token": "",
}

## 📥 Load input data

Safe synthesizer learns the patterns and correlations of an input data set in order to produce synthetic data with similar properties. Use the sample dataset provided or change the following cell to try with your own data.

The sample dataset is of a set of customer default payments. It includes columns of Personally Identifiable Information (PII) such as sex, education level, marriage status, and age. In addition, it contains several billing and payments accounts and a binary indicator of whether the next month's payment would default.

In [ ]:
%pip install ucimlrepo || uv pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
default_of_credit_card_clients = fetch_ucirepo(id=350) 
df = default_of_credit_card_clients.data.original
  

# Display the first few rows of the combined DataFrame
print(df.head()) 

In [ ]:
df

## 🏗️ Create a Safe Synthesizer job

The `SafeSynthesizerBuilder` provides a fluent interface to configure and submit jobs.

This job will:
- Initialize the builder with the NeMo Microservices client.
- Use the loaded DataFrame as the input data source.
- Configure the job to use the specified datastore for model storage.
- Enable automatic replacement of personally identifiable information (PII).
- Enable differential privacy (DP) with a configurable epsilon.
- Use structured generation to enforce the schema during data generation.
- Submit the job to the microservices platform.

In [ ]:
job = (
    SafeSynthesizerBuilder(client)
    .from_data_source(df)
    .with_datastore(datastore_config)
    .with_replace_pii()
    .with_differential_privacy(dp_enabled=True, epsilon=8.0)
    .with_generate(use_structured_generation=True)
    .create_job()
)

print(f"job_id = {job.job_id}")
job.wait_for_completion()

print(f"Job finished with status {job.fetch_status()}")

In [ ]:
# If your notebook shuts down, it's okay, your job is still running on the microservices platform.
# You can get the same job object and interact with it again by uncommenting the following code
# snippet, and modifying it with the job id from the previous cell output.

# from nemo_microservices.beta.safe_synthesizer.sdk.job import SafeSynthesizerJob
# job = SafeSynthesizerJob(job_id="<job id>", client=client)

## 👀 View synthetic data

After the job completes, fetch the generated synthetic dataset.

In [ ]:
# Fetch the synthetic data created by the job
synthetic_df = job.fetch_data()
synthetic_df


## 📊 View evaluation report

An evaluation comparing the synthetic data to the input data is performed automatically.

- Programmatically access key scores (quality and privacy).
- Download the full HTML report with charts and detailed metrics.
- Display the report inline below.

In [ ]:
# Print selected information from the job summary
summary = job.fetch_summary()
print(
    f"Synthetic data quality score (0-10, higher is better): {summary.synthetic_data_quality_score}"
)
print(f"Data privacy score (0-10, higher is better): {summary.data_privacy_score}")


In [ ]:
# Download the full evaluation report to your local machine
job.save_report("evaluation_report.html")

In [ ]:
# Fetch and display the full evaluation report inline
job.display_report_in_notebook()